# Out-of-State-Contributions: National Analysis

How much out-of-state money have candidates nationally raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [27]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


Import contributions data.

In [28]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130104 entries, 0 to 6130103
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(1), int64(2), object(16)
memory usage: 888.6+ MB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type.

In [29]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130104 entries, 0 to 6130103
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 888.6+ MB


## Calculate out-of-state contributions by party and year

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [30]:
contributions_by_party = contributions.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"462,957,682.33"
1,2010,Democratic,out-of-state,"58,240,500.47"
2,2010,Democratic,unknown,"1,540,157.68"
3,2010,Nonpartisan,in-state,"941,726.00"
4,2010,Nonpartisan,out-of-state,"136,753.70"


Pivot dataframe to aggregate each party's data in a single row.

In [31]:
contributions_by_party = pd.pivot_table(contributions_by_party, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party

party         amount                             \
year                                2010                              
in_out_state                    in-state  out-of-state      unknown   
0              Democratic 462,957,682.33 58,240,500.47 1,540,157.68   
1             Nonpartisan     941,726.00    136,753.70   -19,688.72   
2              Republican 636,556,347.66 48,498,147.59   698,087.76   
3             Third-Party   6,077,507.84    650,327.38 1,536,756.63   
4                 Unknown            nan           nan          nan   

                                                                        \
year                   2014                                       2018   
in_out_state       in-state  out-of-state       unknown       in-state   
0            394,123,750.84 68,032,607.74  6,027,913.57 608,404,496.79   
1              1,617,096.45    156,743.05      4,068.00   1,376,332.38   
2            449,095,782.66 73,416,692.06 10,990,228.52 707,153,873.30   
3              9,582,085.85    605,290.08     37,013.76   3,769,391.65   
4                       nan           nan           nan      28,042.01   

                                         
year                                     
in_out_state  out-of-state      unknown  
0            90,699,012.05 8,216,860.79  
1               310,506.30    -3,156.23  
2            66,897,709.39 8,759,998.16  
3               392,743.60   114,439.76  
4                 7,944.09          nan

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [32]:
contributions_by_party.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [33]:
contributions_by_party.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"462,957,682.33","58,240,500.47","1,540,157.68","394,123,750.84","68,032,607.74","6,027,913.57","608,404,496.79","90,699,012.05","8,216,860.79"
1,Nonpartisan,"941,726.00","136,753.70","-19,688.72","1,617,096.45","156,743.05","4,068.00","1,376,332.38","310,506.30","-3,156.23"
2,Republican,"636,556,347.66","48,498,147.59","698,087.76","449,095,782.66","73,416,692.06","10,990,228.52","707,153,873.30","66,897,709.39","8,759,998.16"
3,Third-Party,"6,077,507.84","650,327.38","1,536,756.63","9,582,085.85","605,290.08","37,013.76","3,769,391.65","392,743.60","114,439.76"
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00


Calculate the total contributions by cycle.

In [34]:
contributions_by_party["18_total"] = contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"]
contributions_by_party["14_total"] = contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"]
contributions_by_party["10_total"] = contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"]
contributions_by_party = contributions_by_party[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [35]:
contributions_by_party["pct_18_in_state"] = contributions_by_party["18_in_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_out_of_state"] = contributions_by_party["18_out_of_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_unknown"] = contributions_by_party["18_unknown"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_14_in_state"] = contributions_by_party["14_in_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_unknown"] = contributions_by_party["14_unknown"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_10_in_state"] = contributions_by_party["10_in_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_out_of_state"] = contributions_by_party["10_out_of_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"608,404,496.79","90,699,012.05","8,216,860.79","707,320,369.63","394,123,750.84","68,032,607.74","6,027,913.57","468,184,272.15","462,957,682.33","58,240,500.47","1,540,157.68","522,738,340.48",0.86,0.13,0.01,0.84,0.15,0.01,0.89,0.11,0.00
1,Nonpartisan,"1,376,332.38","310,506.30","-3,156.23","1,683,682.45","1,617,096.45","156,743.05","4,068.00","1,777,907.50","941,726.00","136,753.70","-19,688.72","1,058,790.98",0.82,0.18,-0.00,0.91,0.09,0.00,0.89,0.13,-0.02
2,Republican,"707,153,873.30","66,897,709.39","8,759,998.16","782,811,580.85","449,095,782.66","73,416,692.06","10,990,228.52","533,502,703.24","636,556,347.66","48,498,147.59","698,087.76","685,752,583.01",0.90,0.09,0.01,0.84,0.14,0.02,0.93,0.07,0.00
3,Third-Party,"3,769,391.65","392,743.60","114,439.76","4,276,575.01","9,582,085.85","605,290.08","37,013.76","10,224,389.69","6,077,507.84","650,327.38","1,536,756.63","8,264,591.85",0.88,0.09,0.03,0.94,0.06,0.00,0.74,0.08,0.19
4,Unknown,"28,042.01","7,944.09",0.00,"35,986.10",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,0.22,0.00,nan,nan,nan,nan,nan,nan


## Calculate out-of-state contributions by office and year

Group by year, standardized office and in-vs.-out-of-state contribution status and sum contributions.

In [36]:
contributions_by_office = contributions.groupby(["year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_office.head()

,year,standardized_office,in_out_state,amount
0,2010,GOVERNOR/LIEUTENANT GOVERNOR,in-state,"670,480,358.41"
1,2010,GOVERNOR/LIEUTENANT GOVERNOR,out-of-state,"64,631,813.56"
2,2010,GOVERNOR/LIEUTENANT GOVERNOR,unknown,"2,034,192.94"
3,2010,STATE HOUSE/ASSEMBLY,in-state,"270,786,646.51"
4,2010,STATE HOUSE/ASSEMBLY,out-of-state,"25,432,604.27"


Pivot the dataframe to aggregate each office's data in a single row.

In [37]:
contributions_by_office = pd.pivot_table(contributions_by_office, index=["standardized_office"], columns=["year", "in_out_state"]).reset_index()
contributions_by_office

standardized_office         amount                \
year                                                 2010                 
in_out_state                                     in-state  out-of-state   
0             GOVERNOR/LIEUTENANT GOVERNOR 670,480,358.41 64,631,813.56   
1                     STATE HOUSE/ASSEMBLY 270,786,646.51 25,432,604.27   
2                             STATE SENATE 165,266,258.91 17,461,311.31   

                                                                      \
year                                2014                               
in_out_state      unknown       in-state  out-of-state       unknown   
0            2,034,192.94 387,505,885.44 88,764,738.25 10,238,245.27   
1              985,213.16 293,820,569.32 34,004,551.62  4,650,127.09   
2              735,907.25 173,092,261.04 19,442,043.06  2,170,851.49   

                                                        
year                   2018                             
in_out_state       in-state  out-of-state      unknown  
0            826,578,170.67 92,526,352.88 7,225,705.75  
1            297,070,000.30 41,058,924.09 6,621,126.95  
2            197,083,965.16 24,722,638.46 3,241,309.78

Flatten the resulting dataframe's multi-index columns.

In [38]:
contributions_by_office.columns = ["standardized_office", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_office

,standardized_office,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"670,480,358.41","64,631,813.56","2,034,192.94","387,505,885.44","88,764,738.25","10,238,245.27","826,578,170.67","92,526,352.88","7,225,705.75"
1,STATE HOUSE/ASSEMBLY,"270,786,646.51","25,432,604.27","985,213.16","293,820,569.32","34,004,551.62","4,650,127.09","297,070,000.30","41,058,924.09","6,621,126.95"
2,STATE SENATE,"165,266,258.91","17,461,311.31","735,907.25","173,092,261.04","19,442,043.06","2,170,851.49","197,083,965.16","24,722,638.46","3,241,309.78"


Calculate the total contributions by cycle.

In [39]:
contributions_by_office["18_total"] = contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"]
contributions_by_office["14_total"] = contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"]
contributions_by_office["10_total"] = contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"]
contributions_by_office = contributions_by_office[["standardized_office", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]
contributions_by_office

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total
0,GOVERNOR/LIEUTENANT GOVERNOR,"826,578,170.67","92,526,352.88","7,225,705.75","926,330,229.30","387,505,885.44","88,764,738.25","10,238,245.27","486,508,868.96","670,480,358.41","64,631,813.56","2,034,192.94","737,146,364.91"
1,STATE HOUSE/ASSEMBLY,"297,070,000.30","41,058,924.09","6,621,126.95","344,750,051.34","293,820,569.32","34,004,551.62","4,650,127.09","332,475,248.03","270,786,646.51","25,432,604.27","985,213.16","297,204,463.94"
2,STATE SENATE,"197,083,965.16","24,722,638.46","3,241,309.78","225,047,913.40","173,092,261.04","19,442,043.06","2,170,851.49","194,705,155.59","165,266,258.91","17,461,311.31","735,907.25","183,463,477.47"


Calculate the proportion of in-state, out-of-state and unknown contributions.

In [40]:
contributions_by_office["pct_18_in_state"] = contributions_by_office["18_in_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_out_of_state"] = contributions_by_office["18_out_of_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_unknown"] = contributions_by_office["18_unknown"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"826,578,170.67","92,526,352.88","7,225,705.75","926,330,229.30","387,505,885.44","88,764,738.25","10,238,245.27","486,508,868.96","670,480,358.41","64,631,813.56","2,034,192.94","737,146,364.91",0.89,0.10,0.01
1,STATE HOUSE/ASSEMBLY,"297,070,000.30","41,058,924.09","6,621,126.95","344,750,051.34","293,820,569.32","34,004,551.62","4,650,127.09","332,475,248.03","270,786,646.51","25,432,604.27","985,213.16","297,204,463.94",0.86,0.12,0.02
2,STATE SENATE,"197,083,965.16","24,722,638.46","3,241,309.78","225,047,913.40","173,092,261.04","19,442,043.06","2,170,851.49","194,705,155.59","165,266,258.91","17,461,311.31","735,907.25","183,463,477.47",0.88,0.11,0.01


## Calculate 2018 out-of-state contributions by redistricting role

Filter the contributions data to the 2018 cycle.

In [41]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2322227 entries, 0 to 2322226
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 354.3+ MB


Group by redistricting role and in-vs.-out-of-state contribution status and sum contributions.

In [42]:
contributions_by_redistricting = contributions.groupby(["redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_redistricting

,redistricting_role,in_out_state,amount
0,N,in-state,"221,933,100.96"
1,N,out-of-state,"27,208,974.52"
2,N,unknown,"4,242,389.33"
3,Y,in-state,"1,098,799,035.17"
4,Y,out-of-state,"131,098,940.91"
5,Y,unknown,"12,845,753.15"


Pivot dataframe to aggregate each role's data in a single row.

In [43]:
contributions_by_redistricting = pd.pivot_table(contributions_by_redistricting, index=["redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_redistricting

redistricting_role           amount                             
in_out_state                            in-state   out-of-state       unknown
0                             N   221,933,100.96  27,208,974.52  4,242,389.33
1                             Y 1,098,799,035.17 131,098,940.91 12,845,753.15

Flatten the resulting dataframe's multi-index columns.

In [44]:
contributions_by_redistricting.columns = ["redistricting_role", "in_state", "out_of_state", "unknown"]
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown
0,N,"221,933,100.96","27,208,974.52","4,242,389.33"
1,Y,"1,098,799,035.17","131,098,940.91","12,845,753.15"


Calculate the total contributions by redistricting role.

In [45]:
contributions_by_redistricting["total"] = contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions by redistricting role.

In [46]:
contributions_by_redistricting["pct_in_state"] = contributions_by_redistricting["in_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_out_of_state"] = contributions_by_redistricting["out_of_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_unknown"] = contributions_by_redistricting["unknown"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown,total,pct_in_state,pct_out_of_state,pct_unknown
0,N,"221,933,100.96","27,208,974.52","4,242,389.33","253,384,464.81",0.88,0.11,0.02
1,Y,"1,098,799,035.17","131,098,940.91","12,845,753.15","1,242,743,729.23",0.88,0.11,0.01


## Calculate contributions by candidate status and year

Group by year and candidate status and in-vs.-out-of-state contribution status and sum contributions.

In [47]:
contributions_by_status = contributions.groupby(["year", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_status

,year,standardized_status,in_out_state,amount
0,2010,ADVANCED TO GENERAL,in-state,"867,827,197.38"
1,2010,ADVANCED TO GENERAL,out-of-state,"90,997,477.88"
2,2010,ADVANCED TO GENERAL,unknown,"3,036,295.55"
3,2010,DID NOT ADVANCE,in-state,"238,706,066.45"
4,2010,DID NOT ADVANCE,out-of-state,"16,528,251.26"
5,2010,DID NOT ADVANCE,unknown,"719,017.80"
6,2014,ADVANCED TO GENERAL,in-state,"726,072,398.13"
7,2014,ADVANCED TO GENERAL,out-of-state,"123,317,435.10"
8,2014,ADVANCED TO GENERAL,unknown,"11,792,977.70"
9,2014,DID NOT ADVANCE,in-state,"128,346,317.67"


Pivot dataframe to aggregate each candidate status' data in a single row.

In [48]:
contributions_by_status = pd.pivot_table(contributions_by_status, index=["standardized_status"], columns=["year", "in_out_state"]).reset_index()
contributions_by_status

standardized_status         amount                             \
year                                        2010                              
in_out_state                            in-state  out-of-state      unknown   
0             ADVANCED TO GENERAL 867,827,197.38 90,997,477.88 3,036,295.55   
1                 DID NOT ADVANCE 238,706,066.45 16,528,251.26   719,017.80   

                                                                         \
year                   2014                                        2018   
in_out_state       in-state   out-of-state       unknown       in-state   
0            726,072,398.13 123,317,435.10 11,792,977.70 928,407,219.70   
1            128,346,317.67  18,893,897.83  5,266,246.15 392,324,916.43   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            114,589,399.16 10,984,236.49  
1             43,718,516.27  6,103,905.99

Flatten the resulting dataframe's multi-index columns.

In [49]:
contributions_by_status.columns = ["standardized_status", "10_in_state", "10_out_of_state", "10_unknown",
                                   "14_in_state", "14_out_of_state", "14_unknown",
                                   "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_status

,standardized_status,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,ADVANCED TO GENERAL,"867,827,197.38","90,997,477.88","3,036,295.55","726,072,398.13","123,317,435.10","11,792,977.70","928,407,219.70","114,589,399.16","10,984,236.49"
1,DID NOT ADVANCE,"238,706,066.45","16,528,251.26","719,017.80","128,346,317.67","18,893,897.83","5,266,246.15","392,324,916.43","43,718,516.27","6,103,905.99"


Calculate the total contributions by cycle.

In [50]:
contributions_by_status["18_total"] = contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"]
contributions_by_status["14_total"] = contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"]
contributions_by_status["10_total"] = contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"]
contributions_by_status = contributions_by_status[["standardized_status", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [51]:
contributions_by_status["pct_18_in_state"] = contributions_by_status["18_in_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_out_of_state"] = contributions_by_status["18_out_of_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_unknown"] = contributions_by_status["18_unknown"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_14_in_state"] = contributions_by_status["14_in_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_unknown"] = contributions_by_status["14_unknown"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_10_in_state"] = contributions_by_status["10_in_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_out_of_state"] = contributions_by_status["10_out_of_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,standardized_status,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,ADVANCED TO GENERAL,"928,407,219.70","114,589,399.16","10,984,236.49","1,053,980,855.35","726,072,398.13","123,317,435.10","11,792,977.70","861,182,810.93","867,827,197.38","90,997,477.88","3,036,295.55","961,860,970.81",0.88,0.11,0.01,0.84,0.14,0.01,0.90,0.09,0.00
1,DID NOT ADVANCE,"392,324,916.43","43,718,516.27","6,103,905.99","442,147,338.69","128,346,317.67","18,893,897.83","5,266,246.15","152,506,461.65","238,706,066.45","16,528,251.26","719,017.80","255,953,335.51",0.89,0.10,0.01,0.84,0.12,0.03,0.93,0.06,0.00


## Export the data

In [52]:
writer = pd.ExcelWriter("data/national_analysis.xlsx")
contributions_by_party.to_excel(writer, "contributions_by_party", index=False)
contributions_by_office.to_excel(writer, "contributions_by_office", index=False)
contributions_by_redistricting.to_excel(writer, "contributions_by_redistricting", index=False)
contributions_by_status.to_excel(writer, "contributions_by_status", index=False)
writer.save()